### 1. SQL66 牛客每个人最近的登录日期(一)

* what I do: RANK BY DATE WITHIN PARTITION(GROUP) 在每个分组中来对注册日期 进行排序。 如果我直接在新城生的LOGIN 表格中 来用WHERE 
  会报错， 所以我选择把新生成的LOGIN 当作一张表， 然后从这张表中重新选择

In [ ]:
SELECT s.user_id, s.date FROM
(SELECT user_id, date, 
RANK() OVER(PARTITION BY user_id ORDER BY date DESC) AS rak FROM login) AS s
WHERE s.rak=1

In [ ]:
| 牛客给的解法》 使用 集合函数 与GROUP BY 在一起 这样 我们可以不用担心 必须在GROUP BY 里面选择所有SELECT 中出现的COLUMN 因为
 column 也可以出现在 集合函数中

In [ ]:
# HOW TO do filter or order by within each group? 
SELECT user_id, MAX(date) as d FROM login 
GROUP BY user_id 
ORDER BY user_id ASC

### 2. SQL67 牛客每个人最近的登录日期(二)  较难
牛客每天有很多人登录，请你统计一下牛客每个用户最近登录是哪一天，用的是什么设备.

In [ ]:
| 我的想法 拿到这个题 因为也是多表格 属于份表格的情况 那么我们就要 考虑 从小表格 开始 达成 局部目标， 得到部分想要的数据 再 合并各个
表格 可以用 笛卡尔积 where 或者 JOIN 
我是 先把 每个用户 最近登录的时间 导出做成一个表格， 用到了 RANK在 每个GROUP 里面 排序 这样可以包含 CLIENT?ID 如果用
GROUP BY 的话 那么 我们DATE 放在 MAX 函数里面 但是 CLIENT?ID 不能选 因为我们需要它来找到 设备的名字 但是它如果选了的话 那就是 GROUP
BY USER?ID 和 CLIENT?ID 不是我们想要的 我们就想 对USER?ID 单独 排除重复项
因为同一个 用户也有可能用不同设备 


In [2]:
SELECT user.name as u_n, table2.c_n, table2.date  FROM user,

(SELECT c_rank.user_id, client.name as c_n, c_rank.date FROM client,
(SELECT user_id, client_id, date, RANK() OVER(PARTITION BY user_id ORDER BY date DESC)
 AS c_ran FROM login ) AS c_rank
WHERE c_rank.c_ran=1 and client.id=c_rank.client_id) AS table2
WHERE user.id=table2.user_id 
ORDER BY u_n ASC

SyntaxError: invalid syntax (<ipython-input-2-2ed0045443e8>, line 1)

## SQL68 牛客每个人最近的登录日期(三)
牛客每天有很多人登录，请你统计一下牛客新登录用户的次日成功的留存率
(sqlite里查找某一天的后一天的用法是:date(yyyy-mm-dd, '+1 day')，四舍五入的函数为round，sqlite 1/2得到的不是0.5，得到的是0，只有1*1.0/2才会得到0.5
mysql里查找某一天的后一天的用法是:DATE_ADD(yyyy-mm-dd,INTERVAL 1 DAY)，四舍五入的函数为round)

#### 就是判断他们有没有在第一登录后一天的日期登录过 这就叫次日保存率 
#### 对于每一行 我们 看他是不是第一次登录 所以我们考虑所有用户最近登录的表格
#### 他是的话 那么再对他进行后面一天的查看 但是用CASE 查看的时候我们用uni_login 因为我们不需要看同一个用户在其首次登录的那天重复#### 登录了几次 然后 在考虑 CASE 的判断条件， 对于当前行， 我们不仅要求它是首次登录 就是在首次登录的日期表中， 还要要求它在其首
#### 次登陆后的第二天也登陆了 就是它也在 它的ID 和后一天登录日期也在表格中 那么他就是存留成功客户标记为1 不然就是0 
#### 这样我们对 UNI?LOHGIN 中每一行都有了标记。 然后就是 把这个标记好的表格作为一个表 来从里面计算概率

In [ ]:

SELECT ROUND(SUM(tt.cn)/COUNT(tt.cn),3) FROM 
(SELECT uni_login.user_id,
    SUM(CASE 
    WHEN (uni_login.user_id, uni_login.date) 
    IN (SELECT login.user_id, min(login.date) FROM login GROUP BY login.user_id)
    AND (uni_login.user_id, DATE_ADD(uni_login.date,INTERVAL 1 DAY))
    IN (SELECT login.user_id, login.date FROM login GROUP BY login.user_id, login.date)
    THEN 1 ELSE 0 END ) AS cn
FROM 
(SELECT user_id, date FROM login 
GROUP BY user_id, date) AS uni_login
GROUP BY uni_login.user_id) AS tt

### SQL69 牛客每个人最近的登录日期(四)
要求 求每天 新用户登录个数


In [ ]:
# group by date. within each group 
#SELECT date, COUNT(DISTINCT user_id) FROM login 
#GROUP BY date 
# 1. 分组 每个组里都有 日期和 USER_ID 
# 2.最早的日期里 每个distinct 的 都是第一次登录 之后 登录日期在之前组里出现过的 都是老用户
# 我可以想到 要用到CASE 来计数 CASE 的结果会带来新的一列
# 题解中用到 技术神器 SUM-CASE 所以CASE 会得出一列 SUM 会对这列求和
#SELECT user_id, min(date)FROM login 
#GROUP BY user_id
#2|2020-10-12 用户胡id 2 在最近登录日期是12 号
#3|2020-10-12
#1|2020-10-12
#4|2020-10-14
# 按照 用户分类 每个用户可以在不同日期登录设备 我们删选出它最近登陆的日期
# case when (user_id,date) in 
#    (select user_id,min(date)from login group by user_id)
#    then 1 else 0 end
# CASE 在这是对每一行 进行 删选 当这一行的 USER?ID DATE 是在我们最新登录记录中 那么就计数 1
# 比如 第一行中 2号用户最近登录在12 号 就计数 1 但是当我们到第二行时候 用户2 号 有登陆了 
# 但是这不是他的第一次登录 我们不计算在内 计数为0 .。。 当我们遍历过所有行数 就会得到一个
# 计数的列 全是0 或者1. 我们SUM 它 就是 原始列表 新的 列 计数的最终结果
#SELECT date, 
#CASE
#    WHEN (user_id, date) IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
#    THEN 1 ELSE 0 END 
#FROM login
#2020-10-12|1
#2020-10-12|1
#2020-10-12|1
#2020-10-13|0
#2020-10-13|0
#2020-10-14|0
#2020-10-14|1
#2020-10-15|0
# 上面是只用CASE 的结果就是一行行 来判断是0 还是1 然后形成新的一列
# 用SUM 的话 我们就是 结合 GROUP BY 因为我们要知道 每个日期总的有几个1 这些1 都是第一次登录

#SELECT date, 
#SUM(CASE
#  WHEN (user_id, date) IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
 #  THEN 1 ELSE 0 END) 
#FROM login
#GROUP BY date
#ORDER BY date ASC
#这个方法的弊端在于如果统一用户在最近一天登录多次那么会重复计数。 虽然可以通过 所有测试例子
# 但是还是要对 用户和日期去重 但是不知道怎么做因为是一行一行的判断。 第一行是用户2 第一次登录
# 在12 号 但是第二行又是用户2 登录在12 号 就没办法了 这种情况我们应该给0 

# 有个想法 可以制造出一张表 保函 唯一的 (user_id, date) 所以我们不用原表LOGIN 去用CASE SUM
# 我们用这个去重的新表上做
SELECT date, 
SUM(CASE 
    WHEN (uni_login.user_id, uni_login.date) 
    IN (SELECT login.user_id, min(login.date) FROM login GROUP BY user_id) 
    THEN 1 ELSE 0 END) FROM 
(SELECT user_id, date FROM login 
GROUP BY user_id, date) AS uni_login
GROUP BY uni_login.date 
ORDER BY uni_login.date ASC
